In [12]:
import re

def parse_reverse_line_w_rhyme(text):
    '''
    In this file, only rhyme_pairs are used
    '''
    lines = [line.split() for line in text.split('\n') if line.split()]

    obs_counter = 0
    obs = []
    obs_map = {}
    rhyme_pairs = []

    #99 and 126 do not strictly follow format
    for i in range(154):
        if i+1 == 99 or i+1 == 126: continue
        sonnet_start = i * 15
        if 99 < i+1 < 126: sonnet_start += 1
        elif i+1 > 126: sonnet_start -= 1
        for j in range(3):
            rhyme_pairs.append([lines[sonnet_start+1+j*4][-1], lines[sonnet_start+1+j*4+2][-1]])
            rhyme_pairs.append([lines[sonnet_start+1+j*4+1][-1], lines[sonnet_start+1+j*4+3][-1]])
            for k in range(4):
                obs_elem = []
                line = lines[sonnet_start+1+j*4+k]
                for word in line:
                    word = re.sub(r'[^\w\']', '', word).lower()
                    if word not in obs_map:
                        obs_map[word] = obs_counter
                        obs_counter += 1
                    obs_elem.insert(0, obs_map[word])
                obs.append(obs_elem)
        rhyme_pairs.append([lines[sonnet_start+13][-1], lines[sonnet_start+14][-1]])
        for k in range(2):
            obs_elem = []
            line = lines[sonnet_start+13+k]
            for word in line:
                word = re.sub(r'[^\w\']', '', word).lower()
                if word not in obs_map:
                    obs_map[word] = obs_counter
                    obs_counter += 1
                obs_elem.insert(0, obs_map[word])
            obs.append(obs_elem)
    rhyme_pairs = [[re.sub(r'[^\w\']', '', w).lower() for w in p] for p in rhyme_pairs]
    
    return obs, obs_map, rhyme_pairs

rhyme_pairs = parse_reverse_line_w_rhyme(open('shakespeare.txt', 'r').read())[2]

In [13]:
from poem_utils import *

shake = separate_words(parse_poems('shakespeare.txt'))
spens = separate_words(parse_poems('spenser.txt'))
all_poems = shake+spens
print(all_poems)
word_to_idx, idx_to_word = word_idx_mappings(all_poems)
print((word_to_idx))

def extract_lines(collection):
    lines = []
    for poem in collection:
        for line in poem:
            lines.append(line)
    return lines

shake_lines = extract_lines(shake)
spens_lines = extract_lines(spens)

shake_lines_rev_num = [list(reversed([word_to_idx.get(word.lower()) for word in line])) for line in shake_lines]
spens_lines_rev_num = [list(reversed([word_to_idx.get(word.lower()) for word in line])) for line in spens_lines]
all_lines_rev_num = shake_lines_rev_num + spens_lines_rev_num
print(all_lines_rev_num)

Number of poems found: 153
Number of poems found: 88
[[['From', 'fairest', 'creatures', 'we', 'desire', 'increase'], ['That', 'thereby', "beauty's", 'rose', 'might', 'never', 'die'], ['But', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease'], ['His', 'tender', 'heir', 'might', 'bear', 'his', 'memory'], ['But', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes'], ["Feed'st", 'thy', "light's", 'flame', 'with', 'self-substantial', 'fuel'], ['Making', 'a', 'famine', 'where', 'abundance', 'lies'], ['Thy', 'self', 'thy', 'foe', 'to', 'thy', 'sweet', 'self', 'too', 'cruel'], ['Thou', 'that', 'art', 'now', 'the', "world's", 'fresh', 'ornament'], ['And', 'only', 'herald', 'to', 'the', 'gaudy', 'spring'], ['Within', 'thine', 'own', 'bud', 'buriest', 'thy', 'content'], ['And', 'tender', 'churl', "mak'st", 'waste', 'in', 'niggarding'], ['Pity', 'the', 'world', 'or', 'else', 'this', 'glutton', 'be'], ['To', 'eat', 'the', "world's", 'due', 'by', 'the', 'grave', 'and', 'thee']], [['Wh

In [14]:
from set6hmm import *
hmm_rhyme_15_hidden = unsupervised_HMM(all_lines_rev_num, 15, 10)

epoch 1/10
epoch 2/10
epoch 3/10
epoch 4/10
epoch 5/10
epoch 6/10
epoch 7/10
epoch 8/10
epoch 9/10
epoch 10/10


In [24]:
import numpy as np

def get_state(O, obs, L):
    prob = [O[i][obs] for i in range(len(O))]
    prob = [p/sum(prob) for p in prob]
    return np.random.choice(L, p=prob)

from poem_utils import parse_syllable_dict, line_syllable_count

def get_line_w_ending(hmm, idx_to_word, word_to_idx, end):
    obs = word_to_idx[end]
    init_s = get_state(hmm.O, obs, hmm.L)
    states = [init_s]
    sentence = [end]
    for _ in range(7):
        s = np.random.choice(range(hmm.L), p=hmm.A[states[-1]])
        states.append(s)
        e = np.random.choice(range(hmm.D), p=hmm.O[s])
        sentence.append(idx_to_word[e])
    return ' '.join(sentence).capitalize()

syllable_dict = parse_syllable_dict()

def get_line_w_ending_syllables(hmm, idx_to_word, word_to_idx, end, n_syllables=10):
    line = get_line_w_ending(hmm, idx_to_word, word_to_idx, end)
    while n_syllables not in line_syllable_count(line, syllable_dict):
        line = get_line_w_ending(hmm, idx_to_word, word_to_idx, end)
    return ' '.join(line).capitalize()

def rhyme_sample_sonnet(hmm, idx_to_word, word_to_idx, rhymes, seed=None):
    rng = np.random.default_rng(seed=seed)
    sonnet = []
    for _ in range(3):
        r = rng.choice(len(rhymes), 2)
        end_words = [rhymes[r[0]][0], rhymes[r[1]][0], rhymes[r[0]][1], rhymes[r[1]][1]]
        print(end_words)
        stanza = [get_line_w_ending(hmm, idx_to_word, word_to_idx, end) for end in end_words]
        stanza.append('\n')
        sonnet.append('\n'.join(stanza))
    rhyme_choice = rng.choice(len(rhymes))
    stanza = [get_line_w_ending(hmm, idx_to_word, word_to_idx, rhymes[rhyme_choice][0]), 
              get_line_w_ending(hmm, idx_to_word, word_to_idx, rhymes[rhyme_choice][1])]
    sonnet.append('\n'.join(stanza))
    
    return '\n'.join(sonnet)

In [27]:
print(rhyme_sample_sonnet(hmm_rhyme_15_hidden, idx_to_word, word_to_idx, rhyme_pairs))

['still', 'laws', 'will', 'cause']
['this', 'year', 'amiss', 'appear']
['any', 'frame', 'many', 'same']
Still it fortune true issue fears but i
Laws some one beck i have the the
Will prepare your summer the dignifies but to
Cause dear kindness i be hath down store


This fair to by more right gives curious
Year loves is when the i dread that
Amiss needy lovers in i take if to
Appear her there still true of won when


Any same his foot to one with tender
Frame would so grace may that love's my
Many by her let asked theirs thy that
Same that where and and take in can


Upon right alone in not saphires powers hadst
Moan mad fresh of well show one solace
